# Find Sentinel 1 acquisitions with millisecond precision

## Install dependencies

In [80]:
import sys
!{sys.executable} -m pip install python-dateutil sentinelsat


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


## Import dependencies

In [81]:
from typing import Dict
import datetime
from dateutil.parser import parse
from sentinelsat import SentinelAPI, geojson_to_wkt

## SCIHUB credentials

In [104]:
scihub_username = ""
scihub_password = ""

## Path to STK output

In [82]:
path = "sentinel-1-acquisitions.txt"

## Convenience functions to extract start and stop datetimes

In [83]:
def get_start(line: str):
    line_stripped = line.strip()
    start = parse("-".join(line_stripped.split()[1:5]))
    
    return start


def get_end(line: str):
    line_stripped = line.strip()
    start = parse("-".join(line_stripped.split()[5:9]))
    
    return start

## Search for Sentinel 1 acquisitions, filtering by millisecond precision
The default search API appears to miss acquisitions if datetime interval is less than a minute. 
This function buffers the datetime interval for the search API, and performs a final precise datetime filter in memory.

In [106]:
def search(
    datetime_start: datetime.datetime,
    datetime_end: datetime.datetime,
    producttype: str = "GRD",
    platformname: str = "Sentinel-1",
    return_stac: bool = True,
) -> Dict:
    """
    Search Sentinel Hub
    """
    api = SentinelAPI(scihub_username, scihub_password)

    datetime_start_buffered = datetime_start - datetime.timedelta(minutes=2)
    datetime_end_buffered = datetime_end + datetime.timedelta(minutes=2)
    
    print("Precise datetime range", datetime_start, datetime_end)
    print("Buffered datetime range",datetime_start_buffered, datetime_end_buffered)
    
    results = api.query(
        date=(datetime_start_buffered, datetime_end_buffered), 
        platformname=platformname
    )


    items = [item[1] for item in list(results.items()) if item[1]["beginposition"] <= datetime_start and item[1]["endposition"] >= datetime_start]
    
    if len(items):
        print(f"Sentinel 1 acquisition found for {datetime_start} {datetime_end}")
    else:
        print(f"Sentinel 1 acquisition not found for {datetime_start} {datetime_end}")
    
    
    return items


## Read in the STK output

In [85]:
with open(path) as source:
    lines = source.readlines()

In [86]:
lines

['                                                                                   15 Feb 2023 13:56:03\n',
 'Satellite-SENTINEL-1A_39634-Sensor-sentinel-1a-To-Satellite-STARLINK-2739_48639:  Access Summary Report\n',
 '\n',
 '\n',
 'sentinel-1a-To-STARLINK-2739_48639\n',
 '----------------------------------\n',
 '                  Access        Start Time (UTCG)           Stop Time (UTCG)        Duration (sec)\n',
 '                  ------    ------------------------    ------------------------    --------------\n',
 '                       1     7 Feb 2022 05:54:08.340     7 Feb 2022 05:54:08.457             0.117\n',
 '                       2    13 Feb 2022 11:40:15.318    13 Feb 2022 11:40:15.387             0.070\n',
 '                       3    19 Feb 2022 19:51:11.360    19 Feb 2022 19:51:11.405             0.045\n',
 '                       4    26 Feb 2022 12:56:07.603    26 Feb 2022 12:56:07.660             0.057\n',
 '                       5     5 Mar 2022 06:01:02.927

## Experiments with parsing the STK output

In [94]:
for line in lines[8:]:
    line_stripped = line.strip()
    if line_stripped and line_stripped[0].isdigit():
        print(line_stripped.split())

['1', '7', 'Feb', '2022', '05:54:08.340', '7', 'Feb', '2022', '05:54:08.457', '0.117']
['2', '13', 'Feb', '2022', '11:40:15.318', '13', 'Feb', '2022', '11:40:15.387', '0.070']
['3', '19', 'Feb', '2022', '19:51:11.360', '19', 'Feb', '2022', '19:51:11.405', '0.045']
['4', '26', 'Feb', '2022', '12:56:07.603', '26', 'Feb', '2022', '12:56:07.660', '0.057']
['5', '5', 'Mar', '2022', '06:01:02.927', '5', 'Mar', '2022', '06:01:02.980', '0.053']
['6', '7', 'Mar', '2022', '09:49:34.693', '7', 'Mar', '2022', '09:49:34.755', '0.062']
['7', '9', 'Mar', '2022', '12:00:44.707', '9', 'Mar', '2022', '12:00:44.772', '0.065']
['8', '13', 'Mar', '2022', '16:23:36.253', '13', 'Mar', '2022', '16:23:36.321', '0.069']
['9', '26', 'Mar', '2022', '02:22:48.160', '26', 'Mar', '2022', '02:22:48.456', '0.296']
['10', '9', 'Apr', '2022', '13:08:05.246', '9', 'Apr', '2022', '13:08:05.397', '0.151']
['11', '24', 'Apr', '2022', '02:52:36.911', '24', 'Apr', '2022', '02:52:36.960', '0.049']
['12', '26', 'Apr', '2022', '

In [95]:
for line in lines[8:]:
    line_stripped = line.strip()
    if line_stripped and line_stripped[0].isdigit():
        print(line_stripped.split())

['1', '7', 'Feb', '2022', '05:54:08.340', '7', 'Feb', '2022', '05:54:08.457', '0.117']
['2', '13', 'Feb', '2022', '11:40:15.318', '13', 'Feb', '2022', '11:40:15.387', '0.070']
['3', '19', 'Feb', '2022', '19:51:11.360', '19', 'Feb', '2022', '19:51:11.405', '0.045']
['4', '26', 'Feb', '2022', '12:56:07.603', '26', 'Feb', '2022', '12:56:07.660', '0.057']
['5', '5', 'Mar', '2022', '06:01:02.927', '5', 'Mar', '2022', '06:01:02.980', '0.053']
['6', '7', 'Mar', '2022', '09:49:34.693', '7', 'Mar', '2022', '09:49:34.755', '0.062']
['7', '9', 'Mar', '2022', '12:00:44.707', '9', 'Mar', '2022', '12:00:44.772', '0.065']
['8', '13', 'Mar', '2022', '16:23:36.253', '13', 'Mar', '2022', '16:23:36.321', '0.069']
['9', '26', 'Mar', '2022', '02:22:48.160', '26', 'Mar', '2022', '02:22:48.456', '0.296']
['10', '9', 'Apr', '2022', '13:08:05.246', '9', 'Apr', '2022', '13:08:05.397', '0.151']
['11', '24', 'Apr', '2022', '02:52:36.911', '24', 'Apr', '2022', '02:52:36.960', '0.049']
['12', '26', 'Apr', '2022', '

In [96]:
for line in lines[8:]:
    line_stripped = line.strip()
    if line_stripped and line_stripped[0].isdigit():
        start = get_start(line_stripped)
        end = get_end(line_stripped)
        print(start, end)

2022-02-07 05:54:08.340000 2022-02-07 05:54:08.457000
2022-02-13 11:40:15.318000 2022-02-13 11:40:15.387000
2022-02-19 19:51:11.360000 2022-02-19 19:51:11.405000
2022-02-26 12:56:07.603000 2022-02-26 12:56:07.660000
2022-03-05 06:01:02.927000 2022-03-05 06:01:02.980000
2022-03-07 09:49:34.693000 2022-03-07 09:49:34.755000
2022-03-09 12:00:44.707000 2022-03-09 12:00:44.772000
2022-03-13 16:23:36.253000 2022-03-13 16:23:36.321000
2022-03-26 02:22:48.160000 2022-03-26 02:22:48.456000
2022-04-09 13:08:05.246000 2022-04-09 13:08:05.397000
2022-04-24 02:52:36.911000 2022-04-24 02:52:36.960000
2022-04-26 05:52:24.455000 2022-04-26 05:52:24.516000
2022-04-26 06:41:07.085000 2022-04-26 06:41:07.150000
2022-05-02 22:57:24.588000 2022-05-02 22:57:24.636000
2022-05-02 23:46:02.040000 2022-05-02 23:46:02.087000
2022-05-09 16:50:59.328000 2022-05-09 16:50:59.381000
2022-05-13 22:50:42.873000 2022-05-13 22:50:42.937000
2022-05-18 03:13:38.268000 2022-05-18 03:13:38.362000
2022-06-01 14:38:10.681000 2

## Try search function with the final start, end datetime pair from the STK file
The following returns two Sentinel 1 data products. Both are from the same acquisition. One is OCN, the other is SLC.

In [98]:
result = search(start, end)
result

Precise datetime range 2023-02-01 18:43:57.673000 2023-02-01 18:43:57.740000
Buffered datetime range 2023-02-01 18:41:57.673000 2023-02-01 18:45:57.740000


[{'title': 'S1A_WV_OCN__2SSV_20230201T184357_20230201T190021_047046_05A4B5_B098',
  'link': "https://apihub.copernicus.eu/apihub/odata/v1/Products('e69cde9c-2cae-4bb6-a890-912f47c83aaf')/$value",
  'link_alternative': "https://apihub.copernicus.eu/apihub/odata/v1/Products('e69cde9c-2cae-4bb6-a890-912f47c83aaf')/",
  'link_icon': "https://apihub.copernicus.eu/apihub/odata/v1/Products('e69cde9c-2cae-4bb6-a890-912f47c83aaf')/Products('Quicklook')/$value",
  'summary': 'Date: 2023-02-01T18:43:56.927Z, Instrument: SAR-C SAR, Mode: VV, Satellite: Sentinel-1, Size: 47.55 MB',
  'ondemand': 'false',
  'beginposition': datetime.datetime(2023, 2, 1, 18, 43, 56, 927000),
  'endposition': datetime.datetime(2023, 2, 1, 19, 0, 21, 937000),
  'ingestiondate': datetime.datetime(2023, 2, 1, 20, 28, 10, 231000),
  'missiondatatakeid': 369845,
  'orbitnumber': 47046,
  'lastorbitnumber': 47046,
  'relativeorbitnumber': 74,
  'lastrelativeorbitnumber': 74,
  'filename': 'S1A_WV_OCN__2SSV_20230201T184357_2

In [102]:
result_list = []

for line in lines[8:]:
    print()
    line_stripped = line.strip()
    if line_stripped and line_stripped[0].isdigit():
        start = get_start(line_stripped)
        end = get_end(line_stripped)
        result = search(start, end)
        if len(result):
            result_list.extend(result)


Precise datetime range 2022-02-07 05:54:08.340000 2022-02-07 05:54:08.457000
Buffered datetime range 2022-02-07 05:52:08.340000 2022-02-07 05:56:08.457000
Sentinel 1 acquisition found for 2022-02-07 05:54:08.340000 2022-02-07 05:54:08.457000

Precise datetime range 2022-02-13 11:40:15.318000 2022-02-13 11:40:15.387000
Buffered datetime range 2022-02-13 11:38:15.318000 2022-02-13 11:42:15.387000
Sentinel 1 acquisition not found for 2022-02-13 11:40:15.318000 2022-02-13 11:40:15.387000

Precise datetime range 2022-02-19 19:51:11.360000 2022-02-19 19:51:11.405000
Buffered datetime range 2022-02-19 19:49:11.360000 2022-02-19 19:53:11.405000
Sentinel 1 acquisition not found for 2022-02-19 19:51:11.360000 2022-02-19 19:51:11.405000

Precise datetime range 2022-02-26 12:56:07.603000 2022-02-26 12:56:07.660000
Buffered datetime range 2022-02-26 12:54:07.603000 2022-02-26 12:58:07.660000
Sentinel 1 acquisition not found for 2022-02-26 12:56:07.603000 2022-02-26 12:56:07.660000

Precise datetim

In [103]:
result_list

[{'title': 'S1A_IW_SLC__1SDV_20220207T055356_20220207T055431_041803_04F9BD_1309',
  'link': "https://apihub.copernicus.eu/apihub/odata/v1/Products('ad1461ba-bcd6-49eb-b292-5d4a245a9f65')/$value",
  'link_alternative': "https://apihub.copernicus.eu/apihub/odata/v1/Products('ad1461ba-bcd6-49eb-b292-5d4a245a9f65')/",
  'link_icon': "https://apihub.copernicus.eu/apihub/odata/v1/Products('ad1461ba-bcd6-49eb-b292-5d4a245a9f65')/Products('Quicklook')/$value",
  'summary': 'Date: 2022-02-07T05:53:56.203Z, Instrument: SAR-C SAR, Mode: VV VH, Satellite: Sentinel-1, Size: 9.25 GB',
  'ondemand': 'false',
  'beginposition': datetime.datetime(2022, 2, 7, 5, 53, 56, 203000),
  'endposition': datetime.datetime(2022, 2, 7, 5, 54, 31, 548000),
  'ingestiondate': datetime.datetime(2022, 2, 7, 7, 9, 59, 747000),
  'missiondatatakeid': 326077,
  'orbitnumber': 41803,
  'lastorbitnumber': 41803,
  'relativeorbitnumber': 81,
  'lastrelativeorbitnumber': 81,
  'slicenumber': 1,
  'filename': 'S1A_IW_SLC__1SD